In [4]:
ls 'drive/MyDrive/AIHealthcare/fer2013.csv'

drive/MyDrive/AIHealthcare/fer2013.csv


In [5]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#---------------------------------------------------------------------------------------------------------------------------------
def generate_dataset():
    
    """generate dataset from csv"""
    
    df = pd.read_csv("drive/MyDrive/AIHealthcare/fer2013.csv")
    
    train_samples = df[df['Usage']=="Training"]
    validation_samples = df[df["Usage"]=="PublicTest"]
    test_samples = df[df["Usage"]=="PrivateTest"]
    
    y_train = train_samples.emotion.astype(np.int32).values
    y_valid = validation_samples.emotion.astype(np.int32).values
    y_test = test_samples.emotion.astype(np.int32).values
     
    X_train =np.array([ np.fromstring(image, np.uint8, sep=" ").reshape((48,48)) for image in train_samples.pixels])
    X_valid =np.array([ np.fromstring(image, np.uint8, sep=" ").reshape((48,48)) for image in validation_samples.pixels])
    X_test =np.array([ np.fromstring(image, np.uint8, sep=" ").reshape((48,48)) for image in test_samples.pixels])
    
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

#---------------------------------------------------------------------------------------------------------------------------------
    
def generate_model(lr=0.001):
    
    """training model"""
    
    with tf.device('/gpu:0'):  
        
        model = keras.models.Sequential()
        
        model.add(keras.layers.Conv2D(64,(3,3), input_shape=(48,48, 1), padding="same"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.20))
        
        model.add(keras.layers.Conv2D(128,(5,5), padding='same'))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.20))

        model.add(keras.layers.Conv2D(512,(3,3), padding="same"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.20))
        
        model.add(keras.layers.Conv2D(512,(3,3), padding="same"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.25))
        
        model.add(keras.layers.Conv2D(256,(3,3), activation='relu', padding="same"))
        model.add(keras.layers.Conv2D(128,(3,3), padding='same', activation='relu'))
        model.add(keras.layers.MaxPooling2D())
        model.add(keras.layers.Dropout(0.25))
        
        #model.add(keras.layers.GlobalAveragePooling2D())
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(256))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.Dropout(0.5))
        
        model.add(keras.layers.Dense(512, activation='relu'))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('relu'))
        model.add(keras.layers.Dropout(0.5))
        
        model.add(keras.layers.Dense(7,activation='softmax'))
        
        model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(lr=lr) , metrics=['accuracy'])
        return model

In [6]:
#---------------------------------------------------------------------------------------------------------------------------------
        
if __name__=="__main__":
    
    #df = pd.read_csv("./fer2013/fer2013.csv")
    X_train, y_train, X_valid, y_valid, X_test, y_test =  generate_dataset()
    
    X_train = X_train.reshape((-1,48,48,1)).astype(np.float32)
    X_valid = X_valid.reshape((-1,48,48,1)).astype(np.float32)
    X_test = X_test.reshape((-1,48,48,1)).astype(np.float32)
    
    X_train_std = X_train/255.
    X_valid_std = X_valid/255.
    X_test_std = X_test/255.
    
    model = generate_model(0.01)
    with tf.device("/gpu:0"):
        history = model.fit(X_train_std, y_train,batch_size=128,epochs=35, validation_data=(X_valid_std, y_valid), shuffle=True)

Epoch 1/35
225/225 [==============================] - 47s 57ms/step - loss: 2.2779 - accuracy: 0.2145 - val_loss: 1.8316 - val_accuracy: 0.2469
Epoch 2/35
225/225 [==============================] - 12s 54ms/step - loss: 1.8713 - accuracy: 0.2333 - val_loss: 1.8508 - val_accuracy: 0.2497
Epoch 3/35
225/225 [==============================] - 12s 55ms/step - loss: 1.7772 - accuracy: 0.2674 - val_loss: 1.8761 - val_accuracy: 0.1708
Epoch 4/35
225/225 [==============================] - 12s 55ms/step - loss: 1.6693 - accuracy: 0.3138 - val_loss: 2.2471 - val_accuracy: 0.2494
Epoch 5/35
225/225 [==============================] - 12s 54ms/step - loss: 1.5452 - accuracy: 0.3782 - val_loss: 1.8016 - val_accuracy: 0.3107
Epoch 6/35
225/225 [==============================] - 12s 53ms/step - loss: 1.4879 - accuracy: 0.4012 - val_loss: 1.6910 - val_accuracy: 0.3260
Epoch 7/35
225/225 [==============================] - 12s 53ms/step - loss: 1.4114 - accuracy: 0.4238 - val_loss: 1.5572 - val_accuracy:

In [ ]:
X_test_std[0,:,:,:].reshape([1,48,48,1]).shape

In [ ]:
model.predict(X_test_std[0,:,:,:].reshape([1,48,48,1]))